In [1]:
import pdfplumber

def extract_tables_from_pdf(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_tables = page.extract_tables()
            for table in page_tables:
                merged_table = merge_multiline_columns(table)
                tables.append(merged_table)
    return tables

def merge_multiline_columns(table):
    merged_table = []
    for row in table:
        merged_row = []
        for cell in row:
            if isinstance(cell, list):
                merged_cell = ' '.join(cell)
                merged_row.append(merged_cell)
            else:
                merged_row.append(cell)
        merged_table.append(merged_row)
    return merged_table


In [2]:
import sqlite3
import pandas as pd

def data_ingest(table_name, data: pd.DataFrame):
    with sqlite3.connect("mydatabase.db") as conn:
        data.to_sql(table_name, conn, if_exists="replace", index=False)

def pdf_to_dict(tables):
    table_dict = {}
    for i, rows in enumerate(tables):
        columns = [col.replace("-", "_").replace(" ", "") for col in rows[0]]
        data_rows = rows[1:]
        df = pd.DataFrame(data_rows, columns=columns)
        table_name = f"table_{i + 1}"
        table_dict[table_name] = columns
        data_ingest(table_name, df)
    return table_dict


In [3]:
import sqlite3
import pandas as pd

def data_ingest(table_name, data: pd.DataFrame):
    with sqlite3.connect("mydatabase.db") as conn:
        data.to_sql(table_name, conn, if_exists="replace", index=False)

def pdf_to_dict(tables):
    table_dict = {}
    for i, rows in enumerate(tables):
        columns = [col.replace("-", "_").replace(" ", "") for col in rows[0]]
        data_rows = rows[1:]
        df = pd.DataFrame(data_rows, columns=columns)
        table_name = f"table_{i + 1}"
        table_dict[table_name] = columns
        data_ingest(table_name, df)
    return table_dict


In [14]:
from transformers import pipeline

def generate_sql_query(question, table_name, fields):
    prompt = f"""
    Please generate an SQL query based on the following information:
    Question: {question}
    Table Name: {table_name}
    Fields: {', '.join(fields)}
    """
    nlp = pipeline("text-generation", model="distilgpt2", truncation=True, max_new_tokens=50)
    response = nlp(prompt)
    # Extract the generated SQL query from the response
    generated_text = response[0]["generated_text"]
    sql_query = generated_text.split("SQL query:")[1].strip() if "SQL query:" in generated_text else generated_text.strip()
    print(sql_query)
    return sql_query


In [15]:
def read_sql_query(sql, db):
    with sqlite3.connect(db) as conn:
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        column_names = [description[0] for description in cur.description]
    return column_names, rows

def answer_question(question, table_name, fields):
    sql_query = generate_sql_query(question, table_name, fields)
    column_names, rows = read_sql_query(sql_query, "mydatabase.db")
    return column_names, rows


In [19]:
# Extract tables from PDF
pdf_path = '../Sales_Report_Sample.pdf'
extracted_tables = extract_tables_from_pdf(pdf_path)

# Process tables and ingest data
table_dict = pdf_to_dict(extracted_tables)

# Generate a response to a question
question = "select all the records"
table_name = list(table_dict.keys())[0]
fields = table_dict[table_name]
columns, answer = answer_question(question, table_name, fields)
print("Answer:", answer)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Please generate an SQL query based on the following information:
    Question: select all the records
    Table Name: table_1
    Fields: ProductID, ProductName, SalesDate, UnitsSold, TotalRevenue
        Subject: TableName(in the table format)

Now we are now able to generate the following query:
SELECT ALL records FROM TABLE_1 :
SELECT ALL data FROM TABLE_2 :
SELECT BINDERT INTO


OperationalError: near "Please": syntax error